In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 70 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':70}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from time import time
import timeit #imports timeit module
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('../RiverRoad_Quan_RF.csv')

In [18]:
def XGBC(X, y, gridid):
    start_time = timeit.default_timer()
    kf = KFold(n_splits=5, random_state=2, shuffle=True)
    acc_score = [];
    Truth = [];
    Output = [];
    GRID_ID = pd.DataFrame(columns=['GRID_ID'])

    for train_index , test_index in tqdm(kf.split(df)):
        X_train , X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train , y_test = y.iloc[train_index], y.iloc[test_index]
        train_grid = gridid.iloc[train_index]
        test_grid= gridid.iloc[test_index]

        model =  XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=6 ,random_state=2, use_label_encoder=False, eval_metric='mlogloss')

        model.fit(X_train, y_train)
        pred_values = model.predict(X_test)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        GRID_ID = pd.merge(GRID_ID, test_grid, on='GRID_ID', how='outer')
        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values);

    elapsed = timeit.default_timer() - start_time
    accuracy = np.mean(acc_score)
    
    output_df = pd.concat([GRID_ID, pd.DataFrame(Truth, columns=["Truth"]), pd.DataFrame(Output, columns=["Output"])], axis=1)
    return output_df, elapsed, confusion_matrix(Truth, Output), classification_report(Truth, Output), accuracy

## XGBC with all 60 columns

In [22]:
X = df.drop(['GRID_ID', 'wetland_type', 'wetland', 'FID_RiverRd_AOI'], axis=1)
y = df.FID_RiverRd_AOI
gridid = df.GRID_ID
output_df_60, time_60, cm_60, cr_60, acc_60 = XGBC(X, y, gridid)
print(f"{time_60} seconds")
print(f"{acc_60} accuracy")
print(cm_60)
print(cr_60)

5it [01:15, 15.07s/it]

75.34285479999994 seconds
0.9267215502766089 accuracy
[[2703  441]
 [ 190 5277]]
              precision    recall  f1-score   support

           0       0.93      0.86      0.90      3144
           1       0.92      0.97      0.94      5467

    accuracy                           0.93      8611
   macro avg       0.93      0.91      0.92      8611
weighted avg       0.93      0.93      0.93      8611



## XGBC with Asami's 24 columns

In [23]:
columns = ['qu_dsm_MAX',
'qu_hyddem_MIN',
'qu_dsm_MIN',
'qu_smdem_MAX',
'qu_dem_MIN',
'qu_dsm_SUM',
'qu_dsm_MEAN',
'qu_dsm_RANGE',
'qu_dsm_STD',
'qu_floAcu_MAX',
'qu_floAcu_STD',
'qu_hyddem_MAX',
'qu_dem_MAX',
'qu_floAcu_RANGE',
'qu_curpl_SUM',
'qu_hyddem_SUM',
'qu_curpl_MEAN',
'qu_hyddem_MEAN',
'qu_floAcu_MEAN',
'qu_curpl_STD',
'qu_floAcu_SUM',
'qu_smdem_MIN',
'qu_smdem_SUM',
'qu_curv_STD',
]
X = df[columns]
y = df.FID_RiverRd_AOI
gridid = df.GRID_ID
output_df_24, time_24, cm_24, cr_24, acc_24 = XGBC(X, y, gridid)
print(f"{time_24} seconds")
print(f"{acc_24} accuracy")
print(cm_24)
print(cr_24)

5it [00:35,  7.12s/it]

35.628550800000085 seconds
0.9286951222882596 accuracy
[[2722  422]
 [ 192 5275]]
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      3144
           1       0.93      0.96      0.95      5467

    accuracy                           0.93      8611
   macro avg       0.93      0.92      0.92      8611
weighted avg       0.93      0.93      0.93      8611



## XGBC with Asami's 12 columns

In [24]:
columns = [
'qu_dsm_MAX',
'qu_hyddem_MIN',
'qu_dsm_MIN',
'qu_smdem_MAX',
'qu_dem_MIN',
'qu_dsm_SUM',
'qu_dsm_STD',
'qu_floAcu_MAX',
'qu_floAcu_STD',
'qu_curpl_SUM',
'qu_curpl_STD',
'qu_floAcu_SUM',
]
X = df[columns]
y = df.FID_RiverRd_AOI
gridid = df.GRID_ID
output_df_12, time_12, cm_12, cr_12, acc_12 = XGBC(X, y, gridid)
print(f"{time_12} seconds")
print(f"{acc_12} accuracy")
print(cm_12)
print(cr_12)

5it [00:23,  4.73s/it]

23.685219099999813 seconds
0.930204859713799 accuracy
[[2734  410]
 [ 191 5276]]
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      3144
           1       0.93      0.97      0.95      5467

    accuracy                           0.93      8611
   macro avg       0.93      0.92      0.92      8611
weighted avg       0.93      0.93      0.93      8611



In [25]:
output_df_60.to_csv("../Comparison Data/XGBC60.csv", index=False)
output_df_24.to_csv("../Comparison Data/XGBC24.csv", index=False)
output_df_12.to_csv("../Comparison Data/XGBC12.csv", index=False)